# testing gpu

In [38]:
# import sys
# print("Python executable:", sys.executable)
# print("Environment path:", sys.prefix)

Python executable: C:\Users\light\AppData\Local\Programs\Python\Python312\python.exe
Environment path: C:\Users\light\AppData\Local\Programs\Python\Python312


In [2]:
# import tensorflow as tf
# print("TensorFlow version:", tf.__version__)
# print("GPUs Available: ", tf.config.list_physical_devices('GPU'))

In [32]:
# import tensorflow as tf
# tf.debugging.set_log_device_placement(True)

# # Example operation
# with tf.device('/GPU:0'):
#     a = tf.constant([[1.0, 2.0, 3.0]])
#     b = tf.reduce_sum(a)
#     print("Result:", b)

# getting Libraries version

In [39]:
import sys
import platform
from importlib.metadata import version, PackageNotFoundError

# List of libraries to check
libraries = [
    "scipy", "h5py", "torch", "torchvision", "tensorflow",
    "tensorflow_datasets", "matplotlib", "numpy", "Pillow",
    "keras", "sklearn"
]

# Get Python version and OS details
python_version = sys.version
os_details = platform.platform()

# Get library versions
library_versions = {}
for lib in libraries:
    try:
        library_versions[lib] = version(lib)
    except PackageNotFoundError:
        library_versions[lib] = "Not Installed"

# Print results
print("Environment Details:")
print(f"Python Version: {python_version}")
print(f"Operating System: {os_details}\n")

print("Library Versions:")
for lib, ver in library_versions.items():
    print(f"{lib}: {ver}")


Environment Details:
Python Version: 3.12.4 (tags/v3.12.4:8e8a4ba, Jun  6 2024, 19:30:16) [MSC v.1940 64 bit (AMD64)]
Operating System: Windows-11-10.0.22631-SP0

Library Versions:
scipy: 1.14.0
h5py: 3.11.0
torch: 2.3.1
torchvision: 0.18.1
tensorflow: 2.16.2
tensorflow_datasets: 4.9.7
matplotlib: 3.9.1
numpy: 1.26.4
Pillow: 10.2.0
keras: 3.4.1
sklearn: Not Installed


# testing model

In [4]:
from tensorflow.keras.models import load_model

def test_model(model):
    try:
        model = load_model(f"models/{model}")
        config = model.get_config()  # Attempt to retrieve model configuration
        print("Model configuration exists:", config)
    except Exception as e:
        print("Error:", str(e))

In [5]:
test_model('first.keras')

Model configuration exists: {'name': 'efficientnetb0', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': (None, 224, 224, 3), 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_3'}, 'registered_name': None, 'name': 'input_layer_3', 'inbound_nodes': []}, {'module': 'keras.layers', 'class_name': 'Rescaling', 'config': {'name': 'rescaling_2', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'scale': 0.00392156862745098, 'offset': 0.0}, 'registered_name': None, 'build_config': {'input_shape': [None, 224, 224, 3]}, 'name': 'rescaling_2', 'inbound_nodes': [{'args': ({'class_name': '__keras_tensor__', 'config': {'shape': (None, 224, 224, 3), 'dtype': 'float32', 'keras_history': ['input_layer_3', 0, 0]}},), 'kwargs': {}}]}, {'module': 'keras.layers', 'class_name': 'Normalization', 'config': {'name': 'normalization_1', 'trainable': True, 'dtype

C:\Users\light\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 215 variables whereas the saved optimizer has 428 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [6]:
test_model('my_model.h5')

Error: [Errno 2] Unable to synchronously open file (unable to open file: name = 'models/my_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


# testing request

getting dictionary to match the output number from the model to the name of the car

In [7]:
retrieved_dict = {}
with open('data/label_to_number.txt', 'r') as file:
    for line in file:
        key, value = line.strip().split(': ', 1)
        retrieved_dict[key] = value

Getting the path for the training images

In [8]:
import os
output_file = "data/train_test_split.txt"
path_list = []

if os.path.exists(output_file): # if path exist
    with open(output_file, "r") as f:
        for line in f:
            split_type, path = line.strip().split("\t")
            if int(split_type) == 2:
                path_list.append(path)

print(path_list[:5])

['data/cars196/test/AM General Hummer SUV 2000/000046.jpg', 'data/cars196/test/AM General Hummer SUV 2000/000047.jpg', 'data/cars196/test/AM General Hummer SUV 2000/000048.jpg', 'data/cars196/test/AM General Hummer SUV 2000/000049.jpg', 'data/cars196/test/AM General Hummer SUV 2000/000050.jpg']


Getting label

In [9]:
import re
label_list = []

# Iterate through each entry in the list
for path in path_list:
    # Use regex to find text between backslashes
    match = re.search(r"(train|test)/([^/]+)/", path)
    if match:
        car_name = match.group(2)
        label_list.append(car_name)

print(label_list[:5])

['AM General Hummer SUV 2000', 'AM General Hummer SUV 2000', 'AM General Hummer SUV 2000', 'AM General Hummer SUV 2000', 'AM General Hummer SUV 2000']


In [25]:
import tensorflow as tf
# Function to load and preprocess images and label
def load_and_preprocess_image(image_path, label, target_size=(224, 224)):
    image = tf.io.read_file(image_path) # Load
    image = tf.image.decode_jpeg(image, channels=3) # Decode
    image = tf.image.resize(image, target_size) # Resize
    image = image / 255.0 # Normalize
    image = tf.expand_dims(image, axis=0)
    
    # convert string label to int
    label_number = tf.py_function(func=lookup_label, inp=[label], Tout=tf.int32)
    # Make sure label is scalar
    label_number = tf.reshape(label_number, ())
    
    return image, label_number

def lookup_label(label):
    # Convert Tensor label to Python string
    label_str = label.numpy().decode('utf-8')
    # Look up the label in the dictionary
    return label_to_number[label_str]

In [26]:


print(f'Number of image paths: {len(path_list)}')
print(f'Number of labels: {len(label_list)}')

size = (224, 224)
ds_test = tf.data.Dataset.from_tensor_slices((path_list, label_list))
ds_test = ds_test.map(lambda image_path, label: load_and_preprocess_image(image_path, label, size))

Number of image paths: 8041
Number of labels: 8041


In [31]:
#check labels
for _, label in ds_test.take(1):
    print(f"Label shape: {label.shape}, Label value: {label.numpy()}")

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} NameError: name 'label_to_number' is not defined
Traceback (most recent call last):

  File "C:\Users\light\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 268, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\light\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 146, in __call__
    outputs = self._call(device, args)
              ^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\light\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 153, in _call
    ret = self._func(*args)
          ^^^^^^^^^^^^^^^^^

  File "C:\Users\light\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return fun

In [27]:
# import requests

# url = "http://localhost:5000/predict"

# for img in path_list:
#     input_data = {'input': img}
#     try:
#         response_json = response.json()  # Attempt to parse JSON
#         print(response_json)
#     except requests.JSONDecodeError:
#         print("Response is not JSON:", response.text)

# testing model evaluation

In [29]:
model_list = [
    'models/first.h5',
    'models/first.keras',    
    'models/first-tl.h5',
    'models/first-tl.keras',
    'models/first-uf.h5',
    'models/first-uf.keras',
]
import_Model(model_list)

ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(), output.shape=(1, 196)